# Query ConsDB

A basic notebook to learn about querying the ConsDb

Craig Lage - 09-Sep-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
import pandas as pd
import os
os.environ["no_proxy"] += ",.consdb"
from lsst.summit.utils import ConsDbClient
from lsst.summit.utils.utils import computeCcdExposureId

In [ ]:
client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
print(client.schema())  # list the instruments

In [ ]:
client.schema('lsstcomcam') # List tables for an instrument

In [ ]:
client.schema('lsstcomcam', 'cdb_lsstcomcam.visit1_quicklook') # List values in a table

In [ ]:
client.schema('lsstcomcam', 'cdb_lsstcomcam.exposure') # List values in a table

In [ ]:
# Get a particluar value from a particular visit
data = client.query('SELECT astrom_offset_mean_median from cdb_lsstcomcam.visit1_quicklook where day_obs = 20241029 and seq_num = 98')
data

In [ ]:
# Get all of the values for a particular table and visit
data = client.query('SELECT * from cdb_lsstcomcam.exposure where exposure_id = 2024102900098')
for name in data.colnames:
    print(name, data[name].value[0])

In [ ]:
# Get all of the values for a particular table and visit
data = client.query('SELECT * from cdb_lsstcomcam.visit1_quicklook where visit_id = 2024102900098')
for name in data.colnames:
    print(name, data[name].value[0])

In [ ]:
ccdvisit_id = computeCcdExposureId('latiss', 2024053000592, 0)
print(ccdvisit_id)
data = client.query(f'SELECT * from cdb_latiss.ccdvisit1_quicklook where ccdvisit_id = {ccdvisit_id}')
for name in data.colnames:
    print(name, data[name].value[0])

In [ ]:
# I guess this just isn't populated yet?
data = client.query('SELECT ccdvisit_id from cdb_latiss.ccdvisit1_quicklook where ccdvisit_id > 1')
data

In [ ]:
# I guess this just isn't populated yet?
data = client.query('SELECT visit_id from cdb_latiss.visit1_quicklook where visit_id > 1')
data

In [ ]:
data = client.query('SELECT astrom_offset_mean_median from cdb_lsstcomcam.visit1_quicklook where visit_id = 2024102800083')
data

In [ ]:
data = client.query('SELECT seq_num, obs_start, azimuth, zenith_distance from cdb_lsstcomcamsim.exposure where (seq_num > 0 and seq_num < 10 and day_obs = 20240621)')
data

In [ ]:
data = client.query('SELECT * from cdb_lsstcomcamsim.exposure where exposure_id = 7024062700764')
for name in data.colnames:
    print(name, data[name].value[0])


In [ ]:
# Get a particluar value from a particular visit
query = 'SELECT exp_time, exposure_id, obs_start, obs_end '
query += 'from cdb_lsstcomcam.exposure where day_obs = 20241112 and seq_num > 27 and seq_num < 383'
data = client.query(query)
data

In [ ]:
shutter_open = np.sum(data['exp_time'])
total_time = (Time(data['obs_end'][-1], scale ='tai').jd - Time(data['obs_start'][0], scale='tai').jd) * 86400
print(shutter_open, total_time, f"Percent open = {(shutter_open / total_time * 100.0):.1f}%")